# Build Document Index and Retrieval Augmentation Generation (RAG) with LangChain

---

This workshop uses SageMaker Notebook, and please ensure the kernel is set to **conda_python3**.

### Scenario

In post-call analysis, we must offer guidance on how agents should respond to customer requests or demands. Therefore, we send the transcript as the prompt and search through the documents in the vector store to identify the specific guidance that needs to be extracted to address the agent's requirements.

In this notebook we will demonstrate:

1. Build an index on [Chroma](https://www.trychroma.com/) with a given document from the local directory 

2. Implement Retrieval Augmentation Generation (RAG) with [LangChain](https://python.langchain.com/docs/get_started/introduction.html) and [HuggingFaceEmbeddings](https://huggingface.co/sentence-transformers)


### Contents

- [1. Library Installation](#1.-Library-Installation)
- [2. Query the Endpoint with Testing Prompts](#2.-Query-the-Endpoint-with-Testing-Prompts)
- [3. Retrieval Augmentation Generation with LangChain and Chroma](#3.-Retrieval-Augmentation-Generation-with-LangChain-and-Chroma)

**Note**

This notebook serves as a template so that you can easily replace the example data with your own to build a custom question and answering application.

## 1. Library Installation

In [ ]:
!pip install -U langchain --quiet
!pip install pypdf --quiet
!pip install chromadb --quiet
!pip install nltk --quiet

In [ ]:
import json
import boto3
import sagemaker
from sagemaker.session import Session

# variables
sagemaker_session = Session()
data_bucket = sagemaker.Session().default_bucket()
region = boto3.session.Session().region_name
aws_role = sagemaker_session.get_caller_identity_arn()

# boto3 clients
s3 = boto3.client('s3')
print(f"Region is {region}, IAM Role: {aws_role}, S3 Bucket: {data_bucket}")

## 2. Query the Endpoint with Testing Prompts

To offer guidance to agents on handling customer requests and demands, we begin by asking the LLM to summarize the intent from the transcript.

Please note down the endpoint name of Flan T5 Base in `1-model-selection-and-deployment.ipynb`.

In [ ]:
# TODO: replace with the endpoint name
ENDPOINT_NAME = None # jp-huggingface-text2text-flan-t5-base-XXX

In [ ]:
def query_endpoint_with_json_payload(encoded_json, endpoint_name, content_type="application/json"):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType=content_type, Body=encoded_json
    )
    return response

def parse_response_model_flan_t5(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    generated_text = model_predictions["generated_texts"]
    return generated_text


_MODEL_CONFIG_ = {
    "huggingface-text2text-flan-t5-base": {
        "instance type": "ml.g5.2xlarge",
        "env": {},
        "parse_function": parse_response_model_flan_t5,
        "prompt": """Answer based on context:\n\n{context}\n\n{question}""",
        "endpoint_name": ENDPOINT_NAME
    }
}

In [ ]:
prompt_intent = "Based on the transcript, what's the purpose of the call?"

In [ ]:
question = """
Here is what customer said in the call: 
{transcript}
    
{purpose}
"""

In [ ]:
# sample transcript 2
f = open("transcripts/negative-refund.txt", "r")
transcript_neg_refund = f.read()

In [ ]:
prompt = question.format(transcript=transcript_neg_refund, purpose=prompt_intent)

payload = {
    "text_inputs": prompt,
    "max_length": 100,
    "num_return_sequences": 1,
    "top_k": 10,
    "top_p": 0.95, #0.95,
    "do_sample": True,
}

In [ ]:
for model_id in _MODEL_CONFIG_:
    endpoint_name = _MODEL_CONFIG_[model_id]['endpoint_name']
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = _MODEL_CONFIG_[model_id]["parse_function"](query_response)
    print(f"For model: {model_id}, the generated output is: {generated_texts[0]}\n")

In [ ]:
# keep the response for the next session
customer_ask = generated_texts[0]

## 3. Retrieval Augmentation Generation with LangChain and Chroma

Next, we divide the internal customer-facing guidance into chunks and perform an embedding search to retrieve the target documents. We then prompt the LLM to provide appropriate guidance according to the customer's intent.

In [ ]:
from typing import Dict
import nltk

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import NLTKTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import warnings
warnings.filterwarnings('ignore')

nltk.download('punkt')

**[Step 1]** Split the document into chunks.

In [ ]:
# split docs to chucks
loader = PyPDFLoader("StaffCapacityBuilding.pdf")
docs = loader.load()

text_splitter = NLTKTextSplitter(chunk_size=550)
texts = text_splitter.split_documents(docs)

**[Step 2]** Save the chunks and their embeddings into the vector store. This step takes some time for data ingestion.

In [ ]:
# choose your embedding model
embeddings = HuggingFaceEmbeddings()

# ingest documents into your vector store
vectordb = Chroma.from_documents(texts, embeddings)

**[Step 3]** Create a reusable prompt template.

In [ ]:
class QAContentHandler(LLMContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["generated_texts"][0]

qa_content_handler = QAContentHandler()

In [ ]:
prompt_template = """ 
Given the following text from a document, answer the question to the best of your abilities. Answer only from the provided document,, if you do not know the answer 
just say you don't know. DO NOT make up an answer.

Document: {document}
Question: {question}
Answer:
"""

prompt = PromptTemplate(
    input_variables = ["document", "question"], 
    template = prompt_template)

FLAN_T5_PARAMETERS = {
    "max_length": 100,
    "num_return_sequences": 1,
    "top_k": 1,
    "top_p": 1.0,
    "do_sample": False,
}

**[Step 4]** Chain the model and the prompt together.

In [ ]:
# build a question-answer chain
qa_chain = LLMChain(
    llm = SagemakerEndpoint(
        endpoint_name = ENDPOINT_NAME, # replace with your endpoint name if needed
        region_name = region,
        model_kwargs = FLAN_T5_PARAMETERS,
        content_handler = qa_content_handler
    ),
    prompt = prompt
)

**[Step 5]** Extract the chunks based on similarity search.

In [ ]:
# conduct the similarity search through the vector store
similar_docs = vectordb.similarity_search(question, k=3) 

In [ ]:
# print out the retrieved documents
context_list = [a.page_content for a in similar_docs]
metadata_list = [a.metadata.get('source') for a in similar_docs]
context = "\n\n=============\n\n".join(context_list)
print(context)

**[Step 6]** Get the guidance given by the LLM.

In [ ]:
qa_chain.run({
    'document': context,
    'question': f"Can you tell me more about {customer_ask}?",
})